In [1]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.vgg16 import VGG16
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator,load_img
from keras.models import Sequential
import numpy as np
from glob import glob

In [2]:
IMAGE_SIZE=[224,224]
train = '/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train'
test = '/content/drive/MyDrive/Colab Notebooks/Cotton Disease/test'

In [3]:
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

58900480/58889256 [==============================] - 0s 0us/step


In [4]:
for layer in vgg.layers:
    layer.trainable = False

In [5]:
folders = glob('/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train/*')

In [6]:
folders

['/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train/diseased cotton leaf',
 '/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train/diseased cotton plant',
 '/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train/fresh cotton leaf',
 '/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train/fresh cotton plant']

In [7]:
x = Flatten()(vgg.output)

In [8]:
prediction = Dense(len(folders), activation='softmax')(x)

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)

In [9]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [10]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Cotton Disease/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 1951 images belonging to 4 classes.


In [13]:
test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/Colab Notebooks/Cotton Disease/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 106 images belonging to 4 classes.


In [14]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

/usr/local/lib/python3.7/dist-packages/keras/engine/training.py:1915: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
61/61 [==============================] - 661s 10s/step - loss: 1.3067 - accuracy: 0.6034 - val_loss: 0.2947 - val_accuracy: 0.8962
Epoch 2/20
61/61 [==============================] - 36s 587ms/step - loss: 0.2553 - accuracy: 0.9156 - val_loss: 0.2486 - val_accuracy: 0.9151
Epoch 3/20
61/61 [==============================] - 36s 582ms/step - loss: 0.1621 - accuracy: 0.9465 - val_loss: 0.2081 - val_accuracy: 0.9340
Epoch 4/20
61/61 [==============================] - 36s 583ms/step - loss: 0.1594 - accuracy: 0.9460 - val_loss: 0.1343 - val_accuracy: 0.9528
Epoch 5/20
61/61 [==============================] - 36s 596ms/step - loss: 0.1072 - accuracy: 0.9650 - val_loss: 0.1741 - val_accuracy: 0.9434
Epoch 6/20
61/61 [==============================] - 37s 607ms/step - loss: 0.0827 - accuracy: 0.9756 - val_loss: 0.1978 - val_accuracy: 0.9340
Epoch 7/20
61/61 [==============================] - 36s 585ms/step - loss: 0.0946 - accuracy: 0.9660 - val_loss: 0.1357 - val_accuracy: 0.9434


In [15]:
from tensorflow.keras.models import load_model

model.save('model_vgg16.h5')